In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
import joblib
import traceback
from io import BytesIO

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix,
                             roc_curve, auc, precision_recall_fscore_support)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import linkage, dendrogram

sns.set()
st.set_page_config(page_title="Electricity Consumption — Predictive Analytics (Project V2)", layout="wide")

# -------------------------
# Helpers & compatibility
# -------------------------
def make_onehot():
    try:
        return OneHotEncoder(handle_unknown="ignore", sparse_output=False)
    except TypeError:
        return OneHotEncoder(handle_unknown="ignore", sparse=False)
    except Exception:
        return OneHotEncoder(handle_unknown="ignore")

@st.cache_data
def load_csv(file):
    return pd.read_csv(file)

# Wordcloud helpers (build corpus from many columns + create image)
def build_full_corpus(df, cols=None, numeric_to_bins=True, bins=5, use_unique_for=None):
    """
    Build a corpus string from dataframe columns.
      - cols: list of columns to include (None => all columns)
      - numeric_to_bins: convert numeric columns to binned labels (True recommended)
      - bins: number of quantile bins for numeric columns
      - use_unique_for: list of columns to use only unique values (avoid domination by repeated token)
    """
    if cols is None:
        cols = df.columns.tolist()
    if use_unique_for is None:
        use_unique_for = []

    parts = []
    for c in cols:
        if c not in df.columns:
            continue
        ser = df[c].dropna()
        if ser.empty:
            continue

        # numeric -> bins
        if pd.api.types.is_numeric_dtype(df[c]) and numeric_to_bins:
            try:
                b = pd.qcut(ser, q=bins, duplicates='drop').astype(str)
                parts.append(" ".join([f"{c}_{x}" for x in b.tolist()]))
            except Exception:
                parts.append(" ".join(ser.astype(str).tolist()))
        else:
            if c in use_unique_for:
                vals = ser.astype(str).unique().tolist()
            else:
                vals = ser.astype(str).tolist()
            parts.append(" ".join(vals))

    corpus = " ".join(parts)
    # limit very long corpus to avoid memory/wordcloud issues
    max_chars = 800000
    if len(corpus) > max_chars:
        corpus = corpus[:max_chars]
    return corpus

def make_wordcloud_from_corpus(corpus, width=1200, height=480, max_words=350, background_color="white", colormap='tab20'):
    """Return a PIL image of a WordCloud built from corpus string (or None if empty)."""
    if not corpus or str(corpus).strip() == "":
        return None
    stop = set(STOPWORDS)
    wc = WordCloud(width=width, height=height, background_color=background_color,
                   collocations=False, stopwords=stop, max_words=max_words, colormap=colormap)
    img = wc.generate(corpus)
    return img.to_image()

def safe_numeric_series(s):
    return pd.to_numeric(s, errors="coerce")

def download_bytesio(obj):
    bio = BytesIO()
    joblib.dump(obj, bio)
    bio.seek(0)
    return bio

# -------------------------
# Sidebar: upload & settings
# -------------------------
st.sidebar.title("Data & Settings")
uploaded = st.sidebar.file_uploader("Upload CSV (TG-SPDCL dataset)", type=["csv"])
sample_mode = st.sidebar.checkbox("Train & EDA on sample (faster)", value=True)
sample_frac = st.sidebar.slider("Sampling fraction (for training/EDA)", 0.05, 1.0, 0.2, step=0.05) if sample_mode else 1.0
show_trace = st.sidebar.checkbox("Show tracebacks on error", value=False)
train_full = st.sidebar.checkbox("Force full-data training", value=False)

if not uploaded:
    st.info("Upload your CSV to proceed.")
    st.stop()

# -------------------------
# Load dataframe
# -------------------------
df = load_csv(uploaded)

# -------------------------
# Title & Project header
# -------------------------
st.markdown(
    "<h1 style='color:#0f4c81; margin:0'>Electricity Consumption — Predictive Analytics (Project)</h1>"
    "<p style='color:#6b7280; margin-top:6px'>Supervised: Logistic / KNN / DecisionTree · Unsupervised: KMeans / Agglomerative · EDA & Exports</p>",
    unsafe_allow_html=True,
)
st.write("Dataset shape:", df.shape)

# Expected columns (defaults from your notebook)
NUM_COLS = ['totservices', 'billdservices', 'units', 'load']
CAT_COLS = ['circle', 'division', 'subdivision', 'section', 'area', 'catdesc', 'catcode']

# -------------------------
# Tabs UI
# -------------------------
tabs = st.tabs(["📄 Data", "🔍 EDA", "⚙ Preprocessing", "🤖 Supervised", "📊 Model Comparison", "🔮 Predict (auto)", "🌀 Unsupervised", "💾 Save Models"])

# -------------------------
# TAB: Data
# -------------------------
with tabs[0]:
    st.header("Data Preview")
    st.write("Shape:", df.shape)
    st.dataframe(df.head(200))
    st.write("Columns detected:", list(df.columns))

# -------------------------
# TAB: EDA
# -------------------------
with tabs[1]:
    st.header("Exploratory Data Analysis")

    # WordCloud controls (all/selected cols)
    st.subheader("WordCloud — choose columns (All columns recommended)")
    all_cols = list(df.columns)
    default_cols = ['catdesc'] if 'catdesc' in df.columns else all_cols[:3]
    cols_choice = st.multiselect("Pick columns to include in WordCloud (leave empty to use ALL)", options=all_cols, default=default_cols)
    numeric_to_bins = st.checkbox("Convert numeric columns to binned labels (recommended)", value=True)
    use_unique = st.multiselect("Treat these columns by UNIQUE values only (avoid repetition dominating)", options=all_cols, default=['catdesc'] if 'catdesc' in df.columns else [])

    if st.button("Generate WordCloud"):
        with st.spinner("Building wordcloud — this may take a few seconds..."):
            cols_to_use = cols_choice if len(cols_choice) > 0 else None
            corpus = build_full_corpus(df, cols=cols_to_use, numeric_to_bins=numeric_to_bins, bins=5, use_unique_for=use_unique)
            img = make_wordcloud_from_corpus(corpus, width=1300, height=480, max_words=400, colormap='tab20')
            if img is None:
                st.info("No text found for the selected columns — try different columns.")
            else:
                st.image(img, use_container_width=True)

    # Numeric histograms
    st.subheader("Numeric Histograms")
    num_present = [c for c in NUM_COLS if c in df.columns]
    if num_present:
        cols = st.columns(min(4, len(num_present)))
        for i, c in enumerate(num_present):
            fig = px.histogram(df, x=c, nbins=40, title=c)
            cols[i % len(cols)].plotly_chart(fig, use_container_width=True)
    else:
        st.info("No numeric columns found for histograms.")

    # Boxplots
    st.subheader("Boxplots")
    if num_present:
        cols = st.columns(min(4, len(num_present)))
        for i, c in enumerate(num_present):
            fig, ax = plt.subplots(figsize=(6,2))
            sns.boxplot(x=safe_numeric_series(df[c]).dropna(), ax=ax, color="#f6a821")
            ax.set_title(c)
            cols[i % len(cols)].pyplot(fig)

    # Countplot for chosen categorical
    st.subheader("Countplot (categorical)")
    cats_present = [c for c in df.columns if df[c].dtype == object][:40]
    sel_cat = st.selectbox("Choose categorical column", options=[None] + cats_present)
    if sel_cat:
        vc = df[sel_cat].astype(str).value_counts().nlargest(25)
        fig, ax = plt.subplots(figsize=(8,4))
        sns.barplot(x=vc.values, y=vc.index, ax=ax)
        ax.set_title(f"Top values for {sel_cat}")
        st.pyplot(fig)
    else:
        st.info("Choose a categorical column to see counts.")

    # Correlation heatmap
    st.subheader("Correlation heatmap (numeric)")
    if len(num_present) >= 2:
        fig, ax = plt.subplots(figsize=(8,5))
        sns.heatmap(df[num_present].apply(pd.to_numeric, errors='coerce').corr(), annot=True, cmap="coolwarm", ax=ax)
        st.pyplot(fig)
    else:
        st.info("Not enough numeric columns for correlation heatmap.")

    # Scatter matrix (plotly sample)
    if len(num_present) >= 2:
        st.subheader("Scatter matrix (sampled)")
        cols_for_matrix = num_present[:6]
        sample_n = int(min(2000, max(100, len(df) * (sample_frac if sample_mode else 1.0))))
        samp = df[cols_for_matrix].sample(n=sample_n, random_state=42) if len(df) > sample_n else df[cols_for_matrix]
        samp = samp.dropna()
        if samp.shape[0] > 2:
            fig = px.scatter_matrix(samp, dimensions=cols_for_matrix, height=600)
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.info("Not enough sample rows for scatter matrix.")
    else:
        st.info("Need at least 2 numeric columns for scatter matrix.")

# -------------------------
# TAB: Preprocessing
# -------------------------
with tabs[2]:
    st.header("Preprocessing Setup & Preview (safe)")

    num_present = [c for c in NUM_COLS if c in df.columns]
    cat_present = [c for c in CAT_COLS if c in df.columns]

    st.write("Numeric columns detected:", num_present)
    st.write("Categorical columns detected (expected):", cat_present)

    transformers = []
    if num_present:
        transformers.append(('num', Pipeline([('impute', SimpleImputer(strategy='median')), ('scale', StandardScaler())]), num_present))
    if cat_present:
        transformers.append(('cat', Pipeline([('impute', SimpleImputer(strategy='most_frequent')), ('ohe', make_onehot())]), cat_present))

    if not transformers:
        st.info("No transformers can be created: ensure your dataset includes expected numeric/categorical columns.")
    else:
        preprocess = ColumnTransformer(transformers)
        st.success("Preprocessing pipeline preview created.")

        # SAFE preview transform: coerce numeric & fill categorical before fit_transform
        try:
            preview_cols = num_present + cat_present
            df_preview = df.loc[:, preview_cols].copy()
            # numeric: coerce to numeric
            for c in num_present:
                if c in df_preview.columns:
                    df_preview[c] = pd.to_numeric(df_preview[c], errors='coerce').fillna(df_preview[c].median())
            # categorical: fillna with sentinel
            for c in cat_present:
                if c in df_preview.columns:
                    df_preview[c] = df_preview[c].astype(str).fillna("__MISSING__")
            # fit_transform on a small sample for speed
            sample_n = min(2000, max(100, int(len(df_preview) * 0.05)))
            df_preview_sample = df_preview.sample(n=sample_n, random_state=42) if len(df_preview) > sample_n else df_preview
            X_p = preprocess.fit_transform(df_preview_sample)
            st.write("Preview transform shape:", X_p.shape)
        except Exception as e:
            st.error("Preview transform failed — dataset may have incompatible dtypes or be empty for the selected columns.")
            if show_trace: st.text(traceback.format_exc())

# -------------------------
# TAB: Supervised
# -------------------------
with tabs[3]:
    st.header("Supervised Models — Train & Evaluate (safe & visible)")

    # Choose target (preferred order)
    if 'catdesc' in df.columns and df['catdesc'].nunique() > 1:
        TARGET = 'catdesc'
    elif 'catcode' in df.columns and df['catcode'].nunique() > 1:
        TARGET = 'catcode'
    elif 'units' in df.columns:
        df['units_bin'] = pd.qcut(pd.to_numeric(df['units'].fillna(0)), q=4, labels=False, duplicates='drop')
        TARGET = 'units_bin'
        st.info("Using fallback target: units_bin (quartiles)")
    else:
        TARGET = None
        st.error("No suitable target found (catdesc/catcode/units).")

    if TARGET:
        real_num = [c for c in NUM_COLS if c in df.columns]
        real_cat = [c for c in CAT_COLS if c in df.columns]
        FEATURES = real_cat + real_num
        if not FEATURES:
            st.error("No features available. Check NUM_COLS and CAT_COLS definitions.")
        else:
            st.write("Features to be used:", FEATURES)

            # Choose sample vs full training
            use_full_train = st.checkbox("Force full-data training (may be slow)", value=train_full)
            if use_full_train:
                df_model = df.copy()
            else:
                df_model = df.sample(frac=sample_frac, random_state=42) if sample_mode else df.copy()
                st.info(f"Training on sample fraction {sample_frac:.2f} ({len(df_model)} rows).")

            # Prepare X and y safely
            X = df_model[FEATURES].copy()
            for c in real_num:
                if c in X.columns:
                    X[c] = pd.to_numeric(X[c], errors='coerce').fillna(X[c].median())
            for c in real_cat:
                if c in X.columns:
                    X[c] = X[c].astype(str).fillna("__MISSING__")

            le_target = LabelEncoder()
            y = le_target.fit_transform(df_model[TARGET].astype(str).fillna("__MISSING__"))

            vc = np.bincount(y)
            strat = y if (len(vc) > 1 and vc.min() > 1) else None
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=strat)
            st.write("Train/test shapes:", X_train.shape, X_test.shape)

            # Build preprocess for actual columns
            transformers = []
            if real_num:
                transformers.append(('num', Pipeline([('impute', SimpleImputer(strategy='median')), ('scale', StandardScaler())]), real_num))
            if real_cat:
                transformers.append(('cat', Pipeline([('impute', SimpleImputer(strategy='most_frequent')), ('ohe', make_onehot())]), real_cat))
            preprocess = ColumnTransformer(transformers) if transformers else None

            # session state storage
            if 'models' not in st.session_state:
                st.session_state['models'] = {}
            st.session_state['eval_data'] = {'X_test': X_test, 'y_test': y_test, 'label_encoder': le_target, 'features': FEATURES}

            # Training buttons
            col1, col2, col3 = st.columns(3)
            with col1:
                if st.button("Train Logistic Regression"):
                    try:
                        pipe = Pipeline([('pre', preprocess), ('clf', LogisticRegression(max_iter=1000))])
                        pipe.fit(X_train, y_train)
                        st.session_state['models']['LR'] = pipe
                        st.success("Logistic Regression trained.")
                    except Exception as e:
                        st.error("Training LR failed: " + str(e))
                        if show_trace: st.text(traceback.format_exc())

            with col2:
                if st.button("Train KNN"):
                    try:
                        pipe = Pipeline([('pre', preprocess), ('clf', KNeighborsClassifier(n_neighbors=5))])
                        pipe.fit(X_train, y_train)
                        st.session_state['models']['KNN'] = pipe
                        st.success("KNN trained.")
                    except Exception as e:
                        st.error("Training KNN failed: " + str(e))
                        if show_trace: st.text(traceback.format_exc())

            with col3:
                if st.button("Train Decision Tree"):
                    try:
                        pipe = Pipeline([('pre', preprocess), ('clf', DecisionTreeClassifier(max_depth=6, random_state=42))])
                        pipe.fit(X_train, y_train)
                        st.session_state['models']['DT'] = pipe
                        st.success("Decision Tree trained.")
                    except Exception as e:
                        st.error("Training DT failed: " + str(e))
                        if show_trace: st.text(traceback.format_exc())

            # Show summary if models exist + allow selection to evaluate
            if st.session_state.get('models'):
                st.subheader("Model accuracies (test set)")
                results = {}
                for name, mdl in st.session_state['models'].items():
                    try:
                        preds = mdl.predict(X_test)
                        results[name] = round(accuracy_score(y_test, preds), 4)
                    except Exception as e:
                        results[name] = f"err: {e}"
                res_df = pd.DataFrame.from_dict(results, orient='index', columns=['accuracy'])
                st.table(res_df)

                # Evaluation dropdown
                st.subheader("Detailed Evaluation")
                sel_model_name = st.selectbox("Choose model to inspect", options=list(st.session_state['models'].keys()))
                model = st.session_state['models'][sel_model_name]
                try:
                    y_pred = model.predict(X_test)
                    # Confusion matrix
                    cm = confusion_matrix(y_test, y_pred)
                    fig, ax = plt.subplots(figsize=(6,4))
                    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
                    ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
                    st.pyplot(fig)

                    # classification report (text)
                    st.text("Classification report:")
                    st.text(classification_report(y_test, y_pred, target_names=list(le_target.classes_)))

                    # precision/recall/fscore table + heatmap
                    pr, rc, f1, sup = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=0)
                    pr_df = pd.DataFrame({
                        'precision': pr,
                        'recall': rc,
                        'f1': f1,
                        'support': sup
                    }, index=list(le_target.classes_))
                    st.write("Per-class metrics:")
                    st.dataframe(pr_df.round(3))
                    # heatmap
                    fig2, ax2 = plt.subplots(figsize=(6,4))
                    sns.heatmap(pr_df[['precision','recall','f1']], annot=True, fmt='.2f', cmap='viridis', ax=ax2)
                    ax2.set_title("Per-class precision / recall / f1")
                    st.pyplot(fig2)

                    # ROC plotting if predict_proba exists (binary or OVR multiclass)
                    if hasattr(model, "predict_proba"):
                        st.subheader("ROC / AUC (one-vs-rest for multiclass)")
                        from sklearn.preprocessing import label_binarize
                        classes = np.arange(len(le_target.classes_))
                        y_test_bin = label_binarize(y_test, classes=classes)
                        probs = model.predict_proba(X_test)
                        if y_test_bin.shape[1] == 1:
                            fpr, tpr, _ = roc_curve(y_test, probs[:,1])
                            roc_auc = auc(fpr, tpr)
                            fig3, ax3 = plt.subplots()
                            ax3.plot(fpr, tpr, label=f"AUC={roc_auc:.3f}")
                            ax3.plot([0,1],[0,1], '--', color='gray')
                            ax3.set_xlabel("FPR"); ax3.set_ylabel("TPR"); ax3.legend()
                            st.pyplot(fig3)
                        else:
                            fig4, ax4 = plt.subplots(figsize=(6,4))
                            for i in range(y_test_bin.shape[1]):
                                fpr, tpr, _ = roc_curve(y_test_bin[:,i], probs[:,i])
                                roc_auc = auc(fpr, tpr)
                                ax4.plot(fpr, tpr, label=f"{le_target.classes_[i]} (AUC={roc_auc:.2f})")
                            ax4.plot([0,1],[0,1],'--', color='gray')
                            ax4.set_xlabel("FPR"); ax4.set_ylabel("TPR"); ax4.legend(bbox_to_anchor=(1.05,1))
                            st.pyplot(fig4)
                    else:
                        st.info("Model has no predict_proba(); ROC cannot be plotted.")

                except Exception as e:
                    st.error("Evaluation failed: " + str(e))
                    if show_trace: st.text(traceback.format_exc())

# -------------------------
# TAB: Model Comparison
# -------------------------
with tabs[4]:
    st.header("Model Comparison (visual)")

    if not st.session_state.get('models'):
        st.info("No trained models to compare yet. Train models in the Supervised tab.")
    else:
        eval_data = st.session_state.get('eval_data', {})
        X_test = eval_data.get('X_test')
        y_test = eval_data.get('y_test')
        le_target = eval_data.get('label_encoder')

        comp_rows = []
        for name, mdl in st.session_state['models'].items():
            try:
                preds = mdl.predict(X_test)
                acc = accuracy_score(y_test, preds)
                comp_rows.append({'model': name, 'accuracy': acc})
            except Exception:
                comp_rows.append({'model': name, 'accuracy': np.nan})
        comp_df = pd.DataFrame(comp_rows).set_index('model')
        st.table(comp_df)

        # bar chart
        if not comp_df['accuracy'].isna().all():
            fig = px.bar(comp_df.reset_index(), x='model', y='accuracy', title='Model accuracy comparison', text='accuracy')
            st.plotly_chart(fig, use_container_width=True)

        # show aggregated per-class metrics heatmap across models (if feasible)
        st.subheader("Per-class F1 heatmap across models")
        try:
            f1_table = {}
            for name, mdl in st.session_state['models'].items():
                preds = mdl.predict(X_test)
                _, _, f1, _ = precision_recall_fscore_support(y_test, preds, average=None, zero_division=0)
                f1_table[name] = f1
            f1_df = pd.DataFrame(f1_table, index=list(le_target.classes_))
            fig, ax = plt.subplots(figsize=(8, max(4, 0.5 * len(f1_df))))
            sns.heatmap(f1_df, annot=True, fmt='.2f', cmap='magma', ax=ax)
            ax.set_ylabel("Class"); ax.set_xlabel("Model")
            st.pyplot(fig)
        except Exception as e:
            st.info("Per-class heatmap skipped: " + str(e))
            if show_trace: st.text(traceback.format_exc())

# -------------------------
# TAB: Predict (auto for all models)
# -------------------------
with tabs[5]:
    st.header("Predict — Run predictions for all trained models (auto)")

    if not st.session_state.get('models'):
        st.info("No trained supervised models available. Train them in the Supervised tab.")
    else:
        models = st.session_state['models']
        # Determine features for prediction — best-effort from first model's preprocess
        any_model = next(iter(models.values()))
        features_for_pred = None
        try:
            pre = any_model.named_steps.get('pre')
            if pre is not None and hasattr(pre, 'transformers'):
                features_for_pred = []
                for name, trans, cols in pre.transformers:
                    if isinstance(cols, (list, tuple)):
                        features_for_pred.extend(cols)
        except Exception:
            features_for_pred = None

        if not features_for_pred:
            features_for_pred = [c for c in NUM_COLS if c in df.columns]

        st.write("Features used for prediction (inferred):", features_for_pred)

        # Prepare X_pred safely
        X_pred = df[features_for_pred].copy()
        for c in X_pred.columns:
            if c in NUM_COLS:
                X_pred[c] = pd.to_numeric(X_pred[c], errors='coerce').fillna(X_pred[c].median())
            else:
                X_pred[c] = X_pred[c].astype(str).fillna("__MISSING__")

        # Build panels for each model: show head and download
        cols = st.columns(len(models))
        for col, (name, mdl) in zip(cols, models.items()):
            with col:
                st.subheader(name)
                try:
                    preds = mdl.predict(X_pred)
                    df_out = df.copy()
                    df_out['prediction_' + name] = preds
                    st.dataframe(df_out[[*features_for_pred][:3] + ['prediction_' + name]].head(10))
                    csv_bytes = df_out.to_csv(index=False).encode('utf-8')
                    st.download_button(f"Download predictions ({name})", data=csv_bytes, file_name=f"predictions_{name}.csv", mime="text/csv")
                    st.write("Unique prediction counts:", pd.Series(preds).value_counts().to_dict())
                except Exception as e:
                    st.error(f"Prediction failed for {name}: {e}")
                    if show_trace: st.text(traceback.format_exc())

# -------------------------
# TAB: Unsupervised
# -------------------------
with tabs[6]:
    st.header("Unsupervised — KMeans, Agglomerative, PCA, Pairplots, Dendrogram")

    real_num = [c for c in NUM_COLS if c in df.columns]
    if not real_num:
        st.error("No numeric columns found for unsupervised analysis (e.g., 'units','load').")
    else:
        st.write("Numeric features used:", real_num)

        X_unsup = df[real_num].copy()
        for c in real_num:
            X_unsup[c] = pd.to_numeric(X_unsup[c], errors='coerce').fillna(X_unsup[c].median())

        # Elbow
        st.subheader("Elbow method (inertia vs k)")
        K_range = list(range(2, min(10, max(3, len(df)//10)) + 1))
        inertias = []
        for k in K_range:
            try:
                km_tmp = KMeans(n_clusters=k, random_state=42, n_init=5)
                km_tmp.fit(X_unsup)
                inertias.append(km_tmp.inertia_)
            except Exception:
                inertias.append(np.nan)
        fig, ax = plt.subplots(figsize=(6,3))
        ax.plot(K_range, inertias, marker='o', color="#0f4c81")
        ax.set_xlabel("k"); ax.set_ylabel("Inertia"); ax.set_title("Elbow")
        st.pyplot(fig)

        k = st.slider("Choose k for clustering", 2, min(12, max(2, len(df)//10)), 4)

        # KMeans
        st.subheader("KMeans clustering & PCA")
        try:
            km = KMeans(n_clusters=k, random_state=42, n_init=5)
            labels = km.fit_predict(X_unsup)
            st.write("Cluster counts (KMeans):", pd.Series(labels).value_counts().to_dict())

            pca = PCA(n_components=2)
            proj = pca.fit_transform(X_unsup)
            plot_df = pd.DataFrame({'PCA1': proj[:,0], 'PCA2': proj[:,1], 'cluster': labels.astype(str)})
            fig = px.scatter(plot_df, x='PCA1', y='PCA2', color='cluster', title=f'KMeans k={k} (PCA)')
            st.plotly_chart(fig, use_container_width=True)

            # attach cluster column to main df for pairplot usage
            df['_KM_Cluster'] = labels
        except Exception as e:
            st.error("KMeans failed: " + str(e))
            if show_trace: st.text(traceback.format_exc())

        # Agglomerative
        st.subheader("Agglomerative / Hierarchical clustering")
        try:
            agg = AgglomerativeClustering(n_clusters=k, linkage='ward')
            labels_agg = agg.fit_predict(X_unsup)
            st.write("Cluster counts (Agglomerative):", pd.Series(labels_agg).value_counts().to_dict())

            proj_agg = pca.transform(X_unsup)
            plot_df2 = pd.DataFrame({'PCA1': proj_agg[:,0], 'PCA2': proj_agg[:,1], 'cluster': labels_agg.astype(str)})
            fig2 = px.scatter(plot_df2, x='PCA1', y='PCA2', color='cluster', title=f'Agglomerative k={k} (PCA)')
            st.plotly_chart(fig2, use_container_width=True)

            df['_AGG_Cluster'] = labels_agg
        except Exception as e:
            st.error("Agglomerative failed: " + str(e))
            if show_trace: st.text(traceback.format_exc())

        # Pairplots (your requested sns.pairplot lines) — do it safely (sampling + limited features)
        st.subheader("Pairplots of clusters (sampled & limited columns)")
        try:
            # choose up to 6 numeric features to keep pairplot reasonable
            unsup_features = real_num[:6]  # at most 6
            sample_n = min(800, max(100, int(len(df) * 0.1)))  # sample up to 800 or 10% of data
            df_pair = df[[*unsup_features]].copy().sample(n=sample_n, random_state=42) if len(df) > sample_n else df[[*unsup_features]].copy()
            # add cluster columns if present
            if '_KM_Cluster' in df.columns:
                df_pair['_KM_Cluster'] = df.loc[df_pair.index, '_KM_Cluster']
                st.write("Pairplot for KMeans clusters (may take a moment)...")
                g = sns.pairplot(df_pair, vars=unsup_features, hue="_KM_Cluster", palette="tab10", diag_kind="kde", plot_kws={'s':15})
                st.pyplot(g.fig)
                plt.close(g.fig)
            else:
                st.info("KMeans cluster column missing — run KMeans above to create it.")

            if '_AGG_Cluster' in df.columns:
                df_pair2 = df.loc[df_pair.index, [*unsup_features]].copy()
                df_pair2['_AGG_Cluster'] = df.loc[df_pair2.index, '_AGG_Cluster']
                st.write("Pairplot for Agglomerative clusters (may take a moment)...")
                g2 = sns.pairplot(df_pair2, vars=unsup_features, hue="_AGG_Cluster", palette="Set2", diag_kind="kde", plot_kws={'s':15})
                st.pyplot(g2.fig)
                plt.close(g2.fig)
            else:
                st.info("Agglomerative cluster column missing — run Agglomerative above to create it.")
        except Exception as e:
            st.error("Pairplot generation failed or is too slow for full data: " + str(e))
            if show_trace: st.text(traceback.format_exc())

        # Dendrogram (sampled)
        st.subheader("Dendrogram (sampled)")
        try:
            sample_n2 = min(200, len(X_unsup))
            Z = linkage(X_unsup.sample(sample_n2, random_state=42), method='ward')
            fig, ax = plt.subplots(figsize=(10,4))
            dendrogram(Z, truncate_mode='lastp', p=12, ax=ax)
            ax.set_title("Hierarchical Clustering Dendrogram (sampled)")
            st.pyplot(fig)
        except Exception as e:
            st.info("Dendrogram skipped/failed: " + str(e))
            if show_trace: st.text(traceback.format_exc())

# -------------------------
# TAB: Save models
# -------------------------
with tabs[7]:
    st.header("Save / Download Trained Models")
    if not st.session_state.get('models'):
        st.info("No trained models in session. Train models in the Supervised tab.")
    else:
        for name, mdl in st.session_state['models'].items():
            st.write("Model:", name)
            bio = download_bytesio(mdl)
            st.download_button(f"Download {name}.pkl", data=bio, file_name=f"{name}.pkl")

st.markdown("<div style='margin-top:14px; color:#6b7280'>Dashboard V2 ready — metrics visible, comparison added, pairplots included (sampled & safe). WordCloud now built from multiple columns.</div>", unsafe_allow_html=True)
